# Analyse Parameter von Fit - Boxplot

## Packages

In [1]:
import sys
sys.path.append('../datasets') 
import dataset_long as dsl

sys.path.append('../fitting') 
import paramfitt_singlePatient as pf 

# Model
import numpy as np
import tellurium as te
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
# Dataframe
import pandas as pd

#Estimation

from scipy.optimize import minimize 
from collections import OrderedDict
from scipy.stats import qmc
from scipy.optimize import basinhopping
#Analyse Estimation
from scipy.stats import chi2
from scipy.stats import norm

# Save optimization params
import json
import time

## Boxplot Params analyse

In [2]:
# hat Bounadries von aramfit als y-achsenabschnitt

from paramfitt_singlePatient import get_params_bounds
bounds = get_params_bounds()

# Lade Daten aus dem ersten File
with open ('.././fitting/20230903-08-59-54paras_optimized.json', 'r') as jsonfile: #new, correct update function
#with open("../fitting/20230814-12-52-29paras_optimized.json", "r") as jsonfile:
    fit_PTH = json.load(jsonfile)
df_PTH = pd.DataFrame([r["best_params"] for r in fit_PTH["results"]])
df_PTH = np.power(10, df_PTH)

# Lade Daten aus dem zweiten File
with open ('.././fitting/20230903-08-59-11paras_optimized.json', 'r') as jsonfile:   #new, correct update function
#with open("../fitting/20230814-12-54-32paras_optimized.json", "r") as fp:  #thesis plot, with diysfunctional updae function
#with open("../fitting/20230815-11-08-00paras_optimized.json", "r") as fp:
    fit_nonPTH = json.load(jsonfile)
df_nonPTH = pd.DataFrame([r["best_params"] for r in fit_nonPTH["results"]])
df_nonPTH = np.power(10, df_nonPTH)

# Plotten 

columns = [
        't_ART_add',
        'M',
        'BH_max',
        'k_iE_rupture',
        'k_E_infect',
        'k_M_death',
        'I0_death_iE',
        #'hill',
        't_oiE_death',
        'k_BH',
        'J_oiEdeath_0',
        'Hkt_init',
       ]

df_PTH = df_PTH[columns]
# Plotten der Daten
fig, axes = plt.subplots(2, 6, figsize=(12,6),sharex=False)
axes = axes.ravel()  # Wandelt die 2D-Matrix von Achsen in ein 1D-Array um

for (param, _), ax in zip(df_PTH.items(), axes):  #iteritems() bei ältere pandas verrsion
    data_to_plot = [df_PTH[param].dropna().values, df_nonPTH[param].dropna().values]  # Removing NaN values
    ax.boxplot(data_to_plot, labels = ['PADH', 'nonPADH'],medianprops= dict(color='black'), patch_artist=True, boxprops=dict(color='black', facecolor='lightskyblue'),widths=.6)
    
    ax.set_ylabel(param)
    
    #ax.set_xticklabels()
    ax.ticklabel_format(axis = "y", style="sci",scilimits = (1,3))
    
    # Setze die y-Achsenwerte für den aktuellen Parameter
    ax.set_ylim(bounds[param])

for i in range(11, 12):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.savefig("./figures/params_boxplot.pdf", format="pdf")
plt.show()

df_PTH

FileNotFoundError: [Errno 2] No such file or directory: '.././fitting/20230903-08-59-54paras_optimized.json'

In [ ]:
params = pf.get_params_guess().keys()
ant_model = '.././model/OIE_model.ant'
ant_model = te.loada(ant_model)
old_params ={key:ant_model.getValue(key) for key in params} 

with open ('.././fitting/20230903-08-59-54paras_optimized.json', 'r') as jsonfile:    #r= only read #new with fixed function
    optimize_results_file = json.load(jsonfile, object_pairs_hook=OrderedDict)
new_dict = optimize_results_file["results"][0]["best_params"]
new_params = {key:10**value for key,value in new_dict.items()}



diff = {key: new_params[key] - old_params[key] for key in new_dict.keys()}
#print(diff)

for key in new_dict.keys():

    if old_params[key] == 0:
        print(key, new_params[key], old_params[key], diff[key])
    else:
        print(key, new_params[key], old_params[key], "|", round(diff[key]/old_params[key]*100,0))

#old_params, new_dict, diff

# Simulate the model
#res = ant_model.simulate(0, 40, selections=['time']+ all_values)

Hkt_init 0.4692642294245123 0.45 | 4.0
M 359.7074154115344 40000.0 | -99.0
k_E_infect 1.6212604088747193e-06 1e-06 | 62.0
k_iE_rupture 1.6816754025368148 1.0 | 68.0
k_M_death 48.646858108288136 48.0 | 1.0
I0_death_iE 1.9727041641697967e-06 0.0 1.9727041641697967e-06
t_oiE_death 10.489696740923893 20.0 | -48.0
k_BH 0.0022392072113750507 0.001 | 124.0
J_oiEdeath_0 8500.726069511516 7500.0 | 13.0
BH_max 2.8279845867117963 10.0 | -72.0
t_ART_add 3.3878486215432404 7.0 | -52.0
